In [1]:
import pandas as pd
import sqlite3
import re

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

### 1-Cleaning zipcodes of zip_cbsa file

In [3]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

query = """
SELECT * 
FROM zip_cbsa
"""
zip_cbsa = pd.read_sql(query,db)

db.close() 

In [4]:
zip_cbsa.head()

,zip,cbsa,res_ratio,bus_ratio,oth_ratio,tot_ratio
0,00501,35620,0.0,1.0,0.0,1.0
1,00601,38660,1.0,1.0,1.0,1.0
2,00602,10380,1.0,1.0,1.0,1.0
3,00603,10380,1.0,1.0,1.0,1.0
4,00604,10380,1.0,1.0,1.0,1.0


In [5]:
### Drop columns
zip_cbsa = zip_cbsa.drop(["res_ratio", "bus_ratio", "oth_ratio"], axis=1)

In [6]:
zip_cbsa.dtypes

zip           object
cbsa           int64
tot_ratio    float64
dtype: object

In [7]:
zip_cbsa.describe()

,cbsa,tot_ratio
count,47424.000000,47424.000000
mean,46799.180373,0.831878
std,31193.742860,0.341268
min,10100.000000,0.000027
25%,24060.000000,0.964681
50%,36740.000000,1.000000
75%,48900.000000,1.000000
max,99999.000000,1.000000


In [8]:
### check the zip values
#zip_cbsa["zip"].value_counts()

In [9]:
### Sort values
zip_cbsa = zip_cbsa.sort_values(['zip', 'tot_ratio'], ascending=[False, False])

In [10]:
## Keep only the highest ratio
zip_cbsa = zip_cbsa.drop_duplicates(subset='zip', keep='first')

In [11]:
zip_cbsa.shape

(39451, 3)

In [12]:
zip_cbsa.describe()

,cbsa,tot_ratio
count,39451.000000,39451.000000
mean,46439.739677,0.981053
std,30845.636665,0.068706
min,10100.000000,0.337094
25%,24340.000000,1.000000
50%,36740.000000,1.000000
75%,48260.000000,1.000000
max,99999.000000,1.000000


### Cleaning zipcodes from Nppes file

In [13]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

query = """
SELECT * 
FROM nppes
"""
nppes = pd.read_sql(query,db)

db.close() 

In [14]:
nppes.shape

(115486, 15)

In [15]:
nppes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115486 entries, 0 to 115485
Data columns (total 15 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   npi                  115486 non-null  int64  
 1   entity_type_code     115486 non-null  float64
 2   org_name             24501 non-null   object 
 3   last_name            90982 non-null   object 
 4   first_name           90985 non-null   object 
 5   middle_name          62237 non-null   object 
 6   name_prefix          34723 non-null   object 
 7   name_suffix          2915 non-null    object 
 8   provider_credential  74398 non-null   object 
 9   address_1            115486 non-null  object 
 10  address_2            26571 non-null   object 
 11  city                 115486 non-null  object 
 12  state                115486 non-null  object 
 13  zip                  115486 non-null  float64
 14  taxonomy_code        115486 non-null  object 
dtypes: float64(2), in

In [16]:
#nppes["zip"].value_counts()

In [17]:
### float to object conversion for zip column
nppes['zip'] = nppes['zip'].astype(str)

In [18]:
### Remove .0
nppes['zip'] = nppes['zip'].str.replace('\.0','')

/Users/armelleleguelte/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [19]:
### Some zipcodes are less than 9, add 0 if it's the case
def zip_finder(zips):
    if len(zips)==9:
        return zips[:5]
    if len(zips)==8:
        return '0'+zips[:4]
    if len(zips)==7:
        return '00'+zips[:3]
    if len(zips)==5:
        return zips
    if len(zips)==4:
        return '0'+zips
    if len(zips)==3:
        return '00'+zips
    else:
        return 'no real zip'

In [20]:
nppes["zip"] = nppes.zip.apply(zip_finder)
nppes = nppes[nppes.zip!="no real zip"]

In [21]:
nppes.head()

,npi,entity_type_code,org_name,last_name,first_name,middle_name,name_prefix,name_suffix,provider_credential,address_1,address_2,city,state,zip,taxonomy_code
0,1841293891,1.0,None,GIBBS,ELMER,RICKEY,DR.,None,M.D.,49 CLEVELAND ST 310,None,CROSSVILLE,TN,38555,208600000X
1,1659374601,1.0,None,OBERDICK,WENDY,TIPTON,None,None,MD,105 W STONE DR,STE 1F,KINGSPORT,TN,37660,207Q00000X
2,1134122187,1.0,None,RUDNICKE,CHERYL,DENISE,MRS.,None,CRNP,250 25TH AVE N,STE 412,NASHVILLE,TN,37203,363L00000X
3,1003819046,1.0,None,NYLANDER,BARBARA,H,None,None,M.D.,345 23RD AVE N,SUITE 209,NASHVILLE,TN,37203,207VG0400X
4,1750384780,1.0,None,PERRIGIN,JULIE,A,DR.,None,MD,219 CHURCH ST,None,DICKSON,TN,37055,207Q00000X


In [22]:
#nppes["zip"].value_counts()

### Merge nppes and zip_cbsa

In [23]:
nppes_cbsa = nppes.merge(zip_cbsa, on="zip")

In [24]:
nppes_cbsa.head()

,npi,entity_type_code,org_name,last_name,first_name,middle_name,name_prefix,name_suffix,provider_credential,address_1,address_2,city,state,zip,taxonomy_code,cbsa,tot_ratio
0,1841293891,1.0,None,GIBBS,ELMER,RICKEY,DR.,None,M.D.,49 CLEVELAND ST 310,None,CROSSVILLE,TN,38555,208600000X,18900,1.0
1,1871596403,2.0,"CUMBERLAND MEDICAL CENTER, INC.",None,None,None,None,None,None,421 S MAIN ST,None,CROSSVILLE,TN,38555,282N00000X,18900,1.0
2,1851394241,1.0,None,PATTERSON,LARRY,E.,DR.,None,M.D.,15 IRIS LN,None,CROSSVILLE,TN,38555,207W00000X,18900,1.0
3,1013911577,1.0,None,VARCAK,RONALD,JAMES,DR.,None,D.O.,133 HAYES ST,None,CROSSVILLE,TN,38555,207Q00000X,18900,1.0
4,1639175573,1.0,None,BERRY,PIERRE,KINDALL,DR.,None,D.O.,13 BOB TOLLETT LOOP,None,CROSSVILLE,TN,38555,207Q00000X,18900,1.0


In [25]:
nppes_cbsa.shape

(114912, 17)

### Taxonomy 

In [26]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

query = """
SELECT code, grouping, classification, specialization 
FROM nucc_taxonomy
"""
taxonomy = pd.read_sql(query,db)

db.close() 

In [27]:
taxonomy.head()

,code,grouping,classification,specialization
0,193200000X,Group,Multi-Specialty,None
1,193400000X,Group,Single Specialty,None
2,207K00000X,Allopathic & Osteopathic Physicians,Allergy & Immunology,None
3,207KA0200X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Allergy
4,207KI0005X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Clinical & Laboratory Immunology


### Merge Taxonomy to nppes

In [28]:
nppes_cbsa_tax = nppes_cbsa.merge(taxonomy, left_on='taxonomy_code', right_on='code')

In [29]:
nppes_cbsa_tax.shape

(114912, 21)

In [30]:
nppes_cbsa_tax['entity_type_code'] = nppes_cbsa_tax['entity_type_code'].astype(int)

In [31]:
nppes_cbsa_tax.head()

,npi,entity_type_code,org_name,last_name,first_name,middle_name,name_prefix,name_suffix,provider_credential,address_1,address_2,city,state,zip,taxonomy_code,cbsa,tot_ratio,code,grouping,classification,specialization
0,1841293891,1,None,GIBBS,ELMER,RICKEY,DR.,None,M.D.,49 CLEVELAND ST 310,None,CROSSVILLE,TN,38555,208600000X,18900,1.0,208600000X,Allopathic & Osteopathic Physicians,Surgery,None
1,1487647681,1,None,BROCKMAN,J B,None,DR.,None,MD,100 LANTANA RD STE 202,None,CROSSVILLE,TN,38555,208600000X,18900,1.0,208600000X,Allopathic & Osteopathic Physicians,Surgery,None
2,1013973874,1,None,OLAECHEA,REYNALDO,A,MR.,None,MD,124 HAYES ST,None,CROSSVILLE,TN,38555,208600000X,18900,1.0,208600000X,Allopathic & Osteopathic Physicians,Surgery,None
3,1669424107,1,None,BELL,CHRISTOPHER,M,DR.,None,M.D.,124 HAYES ST,None,CROSSVILLE,TN,38555,208600000X,18900,1.0,208600000X,Allopathic & Osteopathic Physicians,Surgery,None
4,1073526596,1,None,FOX,MARK,ALAN,MR.,None,MD,100 LANTANA RD,SUITE 202,CROSSVILLE,TN,38555,208600000X,18900,1.0,208600000X,Allopathic & Osteopathic Physicians,Surgery,None


### Filter for Nashville cbsa only:

In [32]:
#filter for Nashville CBSA code
nash_nppes = nppes_cbsa_tax[nppes_cbsa_tax.cbsa == 34980]

In [33]:
nash_nppes.shape

(38148, 21)

In [34]:
nash_nppes.head()

,npi,entity_type_code,org_name,last_name,first_name,middle_name,name_prefix,name_suffix,provider_credential,address_1,address_2,city,state,zip,taxonomy_code,cbsa,tot_ratio,code,grouping,classification,specialization
27,1497752489,1,None,COOPER,MARK,None,None,None,M.D.,356 24TH AVE N,SUITE 400,NASHVILLE,TN,37203,208600000X,34980,1.0,208600000X,Allopathic & Osteopathic Physicians,Surgery,None
28,1437156270,1,None,MCDOWELL,JAMES,G,None,JR.,M.D.,356 24TH AVE N,SUITE 400,NASHVILLE,TN,37203,208600000X,34980,1.0,208600000X,Allopathic & Osteopathic Physicians,Surgery,None
29,1447257274,1,None,LYNCH,GEORGE,BRANDON,None,None,M.D.,300 20TH AVE N STE 301,None,NASHVILLE,TN,37203,208600000X,34980,1.0,208600000X,Allopathic & Osteopathic Physicians,Surgery,None
30,1881691780,1,None,GEER,RICHARD,J,None,None,M.D.,356 24TH AVE N,SUITE 400,NASHVILLE,TN,37203,208600000X,34980,1.0,208600000X,Allopathic & Osteopathic Physicians,Surgery,None
31,1205834777,1,None,ROBERTS,JOHN,ROBERT,None,None,M.D.,2400 PATTERSON ST,SUITE 215,NASHVILLE,TN,37203,208600000X,34980,1.0,208600000X,Allopathic & Osteopathic Physicians,Surgery,None


In [35]:
### remove useless columns
nash_nppes = nash_nppes.drop(columns=['code', 'tot_ratio','middle_name', 'name_prefix', 'name_suffix', 'address_2', 'city', 'state', 'cbsa'])

In [36]:
nash_nppes.head()

,npi,entity_type_code,org_name,last_name,first_name,provider_credential,address_1,zip,taxonomy_code,grouping,classification,specialization
27,1497752489,1,None,COOPER,MARK,M.D.,356 24TH AVE N,37203,208600000X,Allopathic & Osteopathic Physicians,Surgery,None
28,1437156270,1,None,MCDOWELL,JAMES,M.D.,356 24TH AVE N,37203,208600000X,Allopathic & Osteopathic Physicians,Surgery,None
29,1447257274,1,None,LYNCH,GEORGE,M.D.,300 20TH AVE N STE 301,37203,208600000X,Allopathic & Osteopathic Physicians,Surgery,None
30,1881691780,1,None,GEER,RICHARD,M.D.,356 24TH AVE N,37203,208600000X,Allopathic & Osteopathic Physicians,Surgery,None
31,1205834777,1,None,ROBERTS,JOHN,M.D.,2400 PATTERSON ST,37203,208600000X,Allopathic & Osteopathic Physicians,Surgery,None


In [37]:
nash_nppes.shape

(38148, 12)

In [38]:
#save as new table in database called nash_nppes
db = sqlite3.connect('../data/nppes_lite.sqlite') #open connection

nash_nppes.to_sql('nash_nppes', db, if_exists = 'append', index = False) #save nashville CBSA nppes

db.close() #close connection

In [ ]:
# create a database or connect to an existing one
#db = sqlite3.connect('../data/nppes_lite.sqlite')
# if you need to edit the database...
#cursor = db.cursor()
# Drop the table and return a line that says that it's gone
#cursor.execute("DROP TABLE nash_nppes")
#print("Table dropped...")

### Explore provider_credential values

In [54]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

query = """
SELECT DISTINCT(provider_credential) 
FROM nash_nppes
WHERE provider_credential LIKE 'M%'
"""
test = pd.read_sql(query,db)

db.close() 

In [55]:
test.shape

(754, 1)

In [56]:
test.provider_credential.unique()

array(['M.D.', 'MD', 'M.D., F.A.C.S.', 'MD/PHD', 'MD, MPH', 'MD, MMS',
       'MD, MBA', 'M. D.', 'M.D., R.V.T', 'M.D', 'M.D,', 'MD PHD',
       'MD, PHD', 'M.D., PH.D.', 'MD, MPH, CPH', 'M.D:', 'MD MPH',
       'M.D, MPH', 'M.S., M.D.', 'M.D. MSPH', 'M.D., MBA',
       'MEDICAL DOCTOR', 'MSN, FNP-BC', 'MD, MS', 'MBBS', 'M.D., M.P.H.',
       'M.D., MSC', 'MD, BSC', 'MD MBA', 'MD, MS, MPH',
       'M.B., B.S., PH.D.,', 'M.D., P.A.', 'MD, MPHTM, CTROPMED',
       'M.D., M.ED.', 'M.D.,  MTR.', 'MD, MSTR', 'MD, FAAP', 'M.D., MPH',
       'M..D.', 'M.D., M.P.H., M.B.A.', 'M.D., M.H.S.', 'MS',
       'M.D. PH.D.', 'MA, BS, CPLC, CHT', 'MS OTR/L', 'M.ED.', 'M.D>',
       'M.ED, BCBA', 'M.A., BCBA', 'M.ED', 'M.D., P.C.', 'MA',
       'MD, PHD, THD', 'M.A.', 'M.S.', 'M.S., BCBA', 'M.D., PHD',
       'M.D., M.S.', 'MD, MCLINEPI, FANZCA', 'M.D., M.ED', 'MD, FRCPC',
       'M.B.B.S; M.D.', 'MBBCH', 'M.D. , PH.D.', 'MPAS, PA-C',
       'MHS, PA-C', 'MED; PA-C', 'MSM, PA-C', 'MSM,PA-C',
       'M.S